In [1]:
import pandas as pd
import pandasai as pai
from pandasai_litellm.litellm import LiteLLM

# Configure LiteLLM to use Ollama
llm = LiteLLM(
    model="ollama/mistral:7b",
    api_base="http://localhost:11434"
)

# Configure PandasAI
pai.config.set({
    "llm": llm,
    "verbose": True,
    "enforce_privacy": False
})

print("✅ PandasAI configured with LiteLLM + Ollama!")

✅ PandasAI configured with LiteLLM + Ollama!


In [2]:
import pandas as pd
import pandasai as pai
df = pd.read_excel(r"D:\taipy\GBM_V1.11.xlsx")
# Cell 3: Clean column names for PandasAI
print("Original column names:")
print(df.columns.tolist())

# Clean column names - remove special characters and spaces
df.columns = df.columns.str.replace(' ', '_').str.replace('[^a-zA-Z0-9_]', '', regex=True)

print("\nCleaned column names:")
print(df.columns.tolist())

# Now create the PandasAI DataFrame
df1 = pai.DataFrame(df)
print("\n✅ PandasAI DataFrame created successfully!")



Original column names:
['SL NO', 'DOCTOR_NAME', 'HOSPITAL_NUMBER', 'PATIENT_NAME', 'CONTACTNO', 'DEATH/ ALIVE (7 AUG 2024)', 'DEATH/ ALIVE (OUTSIDE RX DETAILS)', 'status', 'FOLLOW UP STATUS', 'SURGERY DATE', 'Study end time', 'Calculation Time', 'time_in_days', 'time_in_mos', 'TUMOR SIZE', 'SEX', 'AGE', 'LOCATION', 'LOCATION.1', 'LATARALITY', 'HPR DETAILS', 'HPR', 'GRADE', 'PRESS_DIAGNOSIS', 'SURGERY NAME', 'SURGERY TYPE', 'GFAP', 'IDH1/ IDH2', 'ATRX', 'MGMT', 'Mib Lebeling Index', 'RT DOSE', 'RT MODALITY', 'CCRT', 'RT COMPLETION DATE', 'ADJUVANT ChT', 'START DATE', 'END DATE', 'RECC STATUS', 'OS']

Cleaned column names:
['SL_NO', 'DOCTOR_NAME', 'HOSPITAL_NUMBER', 'PATIENT_NAME', 'CONTACTNO', 'DEATH_ALIVE_7_AUG_2024', 'DEATH_ALIVE_OUTSIDE_RX_DETAILS', 'status', 'FOLLOW_UP_STATUS', 'SURGERY_DATE', 'Study_end_time', 'Calculation_Time', 'time_in_days', 'time_in_mos', 'TUMOR_SIZE', 'SEX', 'AGE', 'LOCATION', 'LOCATION1', 'LATARALITY', 'HPR_DETAILS', 'HPR', 'GRADE', 'PRESS_DIAGNOSIS', 'SURGE

In [3]:
# Check if MCP server packages are available
try:
    import mcp
    print("✅ MCP is available!")
except ImportError:
    print("❌ MCP not installed")
    print("To install: pip install mcp")

✅ MCP is available!


In [4]:
# Since mcp-server-ollama doesn't exist, let's use Ollama directly with better configuration
import requests
import json
import time

def test_ollama_with_better_config():
    """Test Ollama with optimized configuration"""
    
    # Test different models with optimized settings
    models_to_test = [
        {
            "name": "llama3.2:3b",
            "payload": {
                "model": "llama3.2:3b",
                "messages": [{"role": "user", "content": "Hello"}],
                "stream": False,
                "options": {
                    "num_ctx": 512,  # Smaller context
                    "num_thread": 4,  # Limit threads
                    "temperature": 0.1  # Lower temperature for faster response
                }
            }
        },
        {
            "name": "mistral:7b", 
            "payload": {
                "model": "mistral:7b",
                "messages": [{"role": "user", "content": "Hello"}],
                "stream": False,
                "options": {
                    "num_ctx": 512,
                    "num_thread": 4,
                    "temperature": 0.1
                }
            }
        }
    ]
    
    for model_test in models_to_test:
        print(f"\n🔍 Testing {model_test['name']} with optimized settings...")
        
        try:
            start_time = time.time()
            response = requests.post(
                "http://localhost:11434/api/chat",
                json=model_test["payload"],
                timeout=30
            )
            end_time = time.time()
            
            if response.status_code == 200:
                data = response.json()
                content = data.get("message", {}).get("content", "No response")
                print(f"✅ {model_test['name']}: {end_time - start_time:.2f} seconds")
                print(f"�� Response: {content[:100]}...")
                return model_test['name']  # Return the working model
            else:
                print(f"❌ {model_test['name']}: HTTP {response.status_code}")
                
        except Exception as e:
            print(f"❌ {model_test['name']}: Error - {e}")
    
    return None

# Test with optimized settings
best_model = test_ollama_with_better_config()
if best_model:
    print(f"\n✅ Best model: {best_model}")
else:
    print("\n❌ No models working with optimized settings")


🔍 Testing llama3.2:3b with optimized settings...
✅ llama3.2:3b: 8.28 seconds
�� Response: Hello! How can I assist you today?...

✅ Best model: llama3.2:3b


In [ ]:
# Use the best model with optimized PandasAI configuration
if best_model:
    import pandas as pd
    import pandasai as pai
    from pandasai_litellm.litellm import LiteLLM
    
    # Configure with the best model
    llm = LiteLLM(
        model=f"ollama/{best_model}",
        api_base="http://localhost:11434"
    )
    
    # Minimal PandasAI config to avoid memory issues
    pai.config.set({
        "llm": llm,
        "verbose": False,
        "enforce_privacy": False,
        "max_retries": 1,  # Reduce retries
        "enable_cache": False  # Disable cache
    })
    
    print(f"✅ PandasAI configured with {best_model}!")
    
    # Test with simple query
    try:
        print("🔍 Testing medical data analysis...")
        response = df1.chat("What is the median age of the patients?")
        print(f"📊 Response: {response}")
    except Exception as e:
        print(f"❌ Error: {e}")

✅ PandasAI configured with llama3.2:3b!
🔍 Testing medical data analysis...
